In [1]:
import os
import pandas as pd
import numpy as np
import glob
import datetime
from datetime import datetime
import bz2
import re
import glob
from tqdm import tqdm
import random
from datetime import timedelta

# This mutes useless warnings:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)


In [ ]:
#first preprocess the data with:
# This will take a VERY long time, I recommend running in the terminal
# !python dataloader.py

In [3]:
from data_loader import NASAAirportDataset

# this will take a while
train_ds = NASAAirportDataset('KCLT', 'data/preprocess/train')



In [2]:

# Change these times and samples
start = datetime(2022, 9, 30, 0, 0, 0)  # Start of the range
end = datetime(2023, 9, 30, 23, 0)  # End of the range
n_samples = 10
time_samples = [start + timedelta(seconds=random.randint(0, int((end - start).total_seconds()))) for _ in range(n_samples)]
print(time_samples)


[datetime.datetime(2022, 12, 29, 10, 58, 22), datetime.datetime(2022, 12, 26, 14, 5, 54), datetime.datetime(2022, 12, 12, 19, 10, 41), datetime.datetime(2023, 8, 9, 3, 1, 50), datetime.datetime(2023, 8, 30, 21, 33, 16), datetime.datetime(2023, 4, 16, 19, 38, 58), datetime.datetime(2022, 10, 21, 16, 7, 22), datetime.datetime(2023, 6, 10, 8, 26, 1), datetime.datetime(2023, 8, 18, 1, 32, 28), datetime.datetime(2023, 1, 11, 10, 1, 3)]


In [4]:
import torch

for i in time_samples:
    print('Flights at time', i)
    inputs, targets =train_ds[i]
    print(inputs)
    print(targets)
    # for flight_data in train_ds[i]:
    #     train model on one row of (flight_data)



Flights at time 2022-12-29 10:58:22
tensor([[ 0.9208, -1.0000,  1.0000,  ...,  0.9352,  0.9305,  1.0091],
        [ 0.8172, -1.0000,  1.0000,  ...,  0.7938,  0.8532,  0.8548],
        [ 0.6179, -1.0000,  1.0000,  ...,  0.6550,  0.6255,  0.7146],
        ...,
        [ 0.6731, -1.0000,  1.0000,  ...,  0.6776,  0.7239,  0.6869],
        [ 0.7744, -1.0000,  1.0000,  ...,  0.7639,  0.7925,  0.7869],
        [ 0.5642, -1.0000, -1.0000,  ...,  0.5559,  0.5819,  0.5757]])
tensor([0.9208, 0.8172, 0.6179, 0.4673, 0.5728, 0.4651, 0.0973, 0.6412, 0.5324,
        0.9371, 0.4768, 0.5886, 0.2061, 0.8325, 0.6091, 0.6372, 0.0782, 0.1322,
        0.4577, 0.7525, 0.6815, 0.7081, 0.0103, 0.4739, 0.9745, 0.1500, 0.1635,
        0.0631, 0.5494, 0.5823, 0.5731, 0.9490, 0.9633, 0.7194, 0.0823, 0.7504,
        0.6621, 0.4432, 0.6840, 0.5970, 0.7424, 0.8044, 0.7042, 0.5127, 0.5975,
        0.5030, 0.6501, 0.5150, 0.7756, 0.7593, 0.8449, 0.4317, 0.6301, 0.8240,
        0.7851, 0.5627, 0.9360, 0.6252, 0.1205, 0.

KeyboardInterrupt: 